# Dashoard

## You need to install dash: pip install dash

## You can see the dasboard here: http://127.0.0.1:8050/

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Load the health indicators dataset
health_indicators = 'Datasets/ScottishHealthSurveyDataByLocalAuthorities.csv'
health_indicators_df = pd.read_csv(health_indicators)

# Load the life expectancy dataset
life_expectancy = 'Datasets/lifeexpectancy_by_counties.csv'
life_expectancy_df = pd.read_csv(life_expectancy)

# Filter for Scottish areas (councils)
scottish_areas_df = life_expectancy_df[life_expectancy_df['Country'] == 'Scotland']

# Get unique areas and indicators for dropdown options
areas = health_indicators_df['FeatureName'].unique()
indicators = health_indicators_df['Scottish Health Survey Indicator'].unique()

num_areas = len(areas)
print(num_areas)
num_indicators = len(indicators)
print(num_indicators)
print (life_expectancy_df)

In [35]:
# Initialize Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Health Indicators and Life Expectancy Dashboard"),
    html.Label("Select Area:"),
    dcc.Dropdown(
        id='area-dropdown',
        options=[{'label': area, 'value': area} for area in areas],
        value=areas[0]
    ),
    html.Label("Select Health Indicator:"),
    dcc.Dropdown(
        id='indicator-dropdown',
        options=[{'label': indicator, 'value': indicator} for indicator in indicators],
        value=indicators[0]
    ),
    dcc.Graph(id='trend-prediction-graph')
])

@app.callback(
    Output('trend-prediction-graph', 'figure'),
    [Input('area-dropdown', 'value'),
     Input('indicator-dropdown', 'value')]
)
def update_graph(selected_area, selected_indicator):
    # Filter the health indicators data for selected area and indicator
    filtered_health_df = health_indicators_df[(health_indicators_df['FeatureName'] == selected_area) & 
                                              (health_indicators_df['Scottish Health Survey Indicator'] == selected_indicator)].copy()

    # Extract the start year from the DateCode and convert it to a numerical value for health indicator data
    if not filtered_health_df.empty:
        filtered_health_df.loc[:, 'StartYear'] = filtered_health_df['DateCode'].apply(lambda x: int(x.split('-')[0]))

        # Calculate the average value for the selected health indicator in the selected area for each StartYear
        average_health_df = filtered_health_df.groupby('StartYear')['Value'].mean().reset_index()

        # Prepare the data for linear regression model for health indicator
        X_health = average_health_df['StartYear'].values.reshape(-1, 1)
        y_health = average_health_df['Value'].values

        # Create and train the linear regression model for health indicator
        model_health = LinearRegression()
        model_health.fit(X_health, y_health)

        # Predict the next 20 years for health indicator
        future_years_health = np.arange(average_health_df['StartYear'].max() + 1, average_health_df['StartYear'].max() + 21).reshape(-1, 1)
        predictions_health = model_health.predict(future_years_health)
    else:
        predictions_health = []
        average_health_df = pd.DataFrame(columns=['StartYear', 'Value'])
        future_years_health = np.array([]).reshape(-1, 1)

    # Filter the life expectancy data for selected area
    life_expectancy_area_df = scottish_areas_df[scottish_areas_df['Area name'] == selected_area].copy()

    # Extract the start year from the Period and convert it to a numerical value for life expectancy data
    if not life_expectancy_area_df.empty:
        life_expectancy_area_df.loc[:, 'StartYear'] = life_expectancy_area_df['Period'].apply(lambda x: int(x.split(' ')[0]))

        # Calculate the average value for life expectancy in the selected area for each StartYear
        average_life_expectancy_area_df = life_expectancy_area_df.groupby('StartYear')['Life expectancy'].mean().reset_index()

        # Prepare the data for linear regression model for life expectancy
        X_life_expectancy = average_life_expectancy_area_df['StartYear'].values.reshape(-1, 1)
        y_life_expectancy = average_life_expectancy_area_df['Life expectancy'].values

        # Create and train the linear regression model for life expectancy
        model_life_expectancy = LinearRegression()
        model_life_expectancy.fit(X_life_expectancy, y_life_expectancy)

        # Predict the next 20 years for life expectancy
        future_years_life_expectancy = np.arange(average_life_expectancy_area_df['StartYear'].max() + 1, average_life_expectancy_area_df['StartYear'].max() + 21).reshape(-1, 1)
        predictions_life_expectancy = model_life_expectancy.predict(future_years_life_expectancy)
    else:
        predictions_life_expectancy = []
        average_life_expectancy_area_df = pd.DataFrame(columns=['StartYear', 'Life expectancy'])
        future_years_life_expectancy = np.array([]).reshape(-1, 1)

    # Create figure with subplots for health indicator and life expectancy trends and predictions
    fig = {
        'data': [
            {'x': average_health_df['StartYear'], 'y': average_health_df['Value'], 'type': 'scatter', 'name': f'{selected_indicator} Historical Data'},
            {'x': future_years_health.flatten(), 'y': predictions_health, 'type': 'scatter', 'name': f'{selected_indicator} Predictions'},
            {'x': average_life_expectancy_area_df['StartYear'], 'y': average_life_expectancy_area_df['Life expectancy'], 'type': 'scatter', 'name': 'Life Expectancy Historical Data'},
            {'x': future_years_life_expectancy.flatten(), 'y': predictions_life_expectancy, 'type': 'scatter', 'name': 'Life Expectancy Predictions'}
        ],
        'layout': {
            'title': f'Trend and Predictions of {selected_indicator} and Life Expectancy in {selected_area}',
            'xaxis': {'title': 'Year'},
            'yaxis': {'title': 'Value'},
            'legend': {'x': 1, 'y': 1}
        }
    }

    return fig

# Run the app in Jupyter notebook
app.run(mode='inline')
